# CUDA exercises

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import pathlib
from matplotlib import rcParams
import itertools
%matplotlib inline

from IPython.display import display

In [ ]:
def csv_parse(path, header=False, sep=";"):
    if header:
        df = pd.read_csv(path, sep=sep, names=header)
    else:
        df = pd.read_csv(path, sep=sep)
    #display(df.head())
    return df

def load_glob(glob, concat=True):
    liste = [pd.read_csv(file) for file in glob]
    name = [file for file in glob]
    repr(name)
    if concat:
        return pd.concat(liste)
    return liste, name

In [ ]:
def transform_to_ms(df, cols):
    return df[cols]*1e3


# Ex7.1

In [ ]:
def print_styling():

    plt.rc('figure', figsize=(12,8)) 
    plt.style.use("seaborn")
    sns.set_palette("Paired")
    SMALL_SIZE = 15
    MEDIUM_SIZE = 18
    BIGGER_SIZE = 26

    plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
    plt.rc('lines', linewidth=2)

    plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
    plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
    plt.rc('axes', titlesize=MEDIUM_SIZE)     # fontsize of the axes title
    plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
    
    plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
    plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title
print_styling()

In [ ]:
# Get the data
cwd = pathlib.Path(os.getcwd())
data_subdir = cwd.parent / "data"
plot_dir = cwd / "out"
data_subdir, plot_dir

In [ ]:
project = {
    "boost": {"OpenCL": pd.DataFrame()},
    "mykernel": {
        "OpenCL": pd.DataFrame(),
        "CUDA": pd.DataFrame()
    },
    "thrust": {"CUDA": pd.DataFrame()},
    "vexcl": {"OpenCL": pd.DataFrame()},
    "viennacl": {
        "OpenCL": pd.DataFrame(),
        "CUDA": pd.DataFrame()
    }
}

all_data = []
all_names = []
for lib in project:
    for imp in project[lib]:
        imp_short = "ocl" if imp=="OpenCL" else "cuda"
        regex_pattern = f"ph_data_{lib}_{imp_short}.csv"
        #print(regex_pattern)
        files = list(data_subdir.glob(regex_pattern))
        #print(files)
        if len(files) > 1:
            print("There shouldn't be more than 1 file...")
        file = files[0]
        df = csv_parse(file)
        project[lib][imp] = df
        all_data.append(df)
        all_names.append(f"{lib} / {imp}")

df_ref = project["mykernel"]["OpenCL"]
for lib in project:
    for imp in project[lib]:
        project[lib][imp]["Speedup"] =  df_ref["dot_time"] / project[lib][imp]["dot_time"]

all_names
#project

In [ ]:
df_ref

In [ ]:
fig, axs = plt.subplots(ncols=2, figsize=(12,8))
axs = axs.ravel()
logx, logy = True, True
if (logx and logy):
    plot_format = "loglog"
if (logx and not logy):
    plot_format = "logx"
if (not logx and logy):
    plot_format = "logy"
else:
    plot_format = "linear"

for df, label in zip(all_data, all_names):
    if "CUDA" in label:
        df.plot(x="N", y="dot_time", label=label, logx=logx, logy=logy, ax=axs[0]);
    else:
        df.plot(x="N", y="dot_time", label=label, logx=logx, logy=logy, ax=axs[1]);

fig.suptitle("Runtime: Dot-product in different libraries")
titles = ["Backend: CUDA", "Backend: OpenCL"]
for ax, title in zip(axs, titles):
    ax.set_title(title)
axs[0].set_ylabel("runtime[s]")


plt.tight_layout()
fig.savefig(plot_dir / "runtimes_per_backend.pdf", bbox_inches="tight", pad_inches=0.2)

In [ ]:
fig, axs = plt.subplots(ncols= 2, figsize=(12,8))
axs = axs.ravel()
logx, logy = True, True
if (logx and logy):
    plot_format = "loglog"
if (logx and not logy):
    plot_format = "logx"
if (not logx and logy):
    plot_format = "logy"
else:
    plot_format = "linear"

for df, label in zip(all_data, all_names):
    ls = "-" if "CUDA" in label else "--"
    df.plot(x="N", y="dot_time", label=label, ls=ls, logx=logx, logy=logy, ax=axs[0]);
    df.plot(x="N", y="Speedup", label=label, ls=ls, logx=logx, logy=logy, ax=axs[1]);

fig.suptitle("Benchmark: Dot-product in different libraries")
titles = ["Runtimes [s]", "Speedup"]
for ax, title in zip(axs, titles):
    ax.set_title(title)
axs[0].set_ylabel("runtime [s]")
axs[1].set_ylabel("speedup")


plt.tight_layout()
fig.savefig(plot_dir / "runtimes_speedup.pdf", bbox_inches="tight", pad_inches=0.2)